## Importing Necessary Libraries

In [ ]:
from pandas.tseries.offsets import BDay
import requests
from datetime import timedelta
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


## Identifying Outliers in last 12 months

In [ ]:
def fetch_data(url):
    response = requests.get(url)
    if response.status_code != 200:
        print("Error fetching data:", response.status_code, response.text)
        return None
    data = response.json()
    if 'results' not in data:
        print("No 'results' key in response:", data)
        return None
    return data

def calculate_daily_returns(df, prev_close=None):
    if prev_close is not None:
        df.loc[df.index[0], 'prev_close'] = prev_close
    else:
        df['prev_close'] = df['c'].shift(1)
    df['daily_return'] = (df['c'] - df['prev_close']) / df['prev_close']
    df['abs_daily_return'] = df['daily_return'].abs()
    return df

def get_top_outliers(df, n=10):
    return df.nlargest(n, 'abs_daily_return')

def update_outliers_list(current_df, historical_outliers_df, real_time_outliers_df, n=10):
    if 'source' not in current_df.columns:
        current_df['source'] = 'real-time'
    combined_df = pd.concat([historical_outliers_df, current_df])
    updated_outliers_df = combined_df.nlargest(n, 'abs_daily_return')
    updated_historical_outliers_df = updated_outliers_df[updated_outliers_df['source'] == 'historical']
    updated_real_time_outliers_df = updated_outliers_df[updated_outliers_df['source'] == 'real-time']
    return updated_historical_outliers_df, updated_real_time_outliers_df

def convert_timestamps(df):
    df['date'] = pd.to_datetime(df['t'], unit='ms')
    df.drop(columns=['t'], inplace=True)
    return df

# API key and endpoints
api_key = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'
today = pd.Timestamp.now().date()
start_date = today - pd.DateOffset(years=1)
start_date_formatted = start_date.strftime('%Y-%m-%d')
end_date = today - pd.DateOffset(days=1)
end_date_formatted = end_date.strftime('%Y-%m-%d')
symbol = 'C:USDCHF'
historical_url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{start_date_formatted}/{end_date_formatted}?adjusted=true&sort=asc&apiKey={api_key}'
real_time_url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{today}/{today}?adjusted=true&sort=asc&apiKey={api_key}'

# Fetch and process historical data
historical_data = fetch_data(historical_url)
if historical_data:
    historical_df = pd.DataFrame(historical_data['results'])
    historical_df = convert_timestamps(historical_df)
    historical_df = calculate_daily_returns(historical_df)
    historical_df['source'] = 'historical'
    historical_outliers_df = get_top_outliers(historical_df)
else:
    print("Failed to fetch or process historical data.")

# Fetch and process real-time data
real_time_data = fetch_data(real_time_url)
if real_time_data and 'results' in real_time_data:
    real_time_df = pd.DataFrame(real_time_data['results'])
    real_time_df = convert_timestamps(real_time_df)
    # Use the last close from historical data
    last_close = historical_df['c'].iloc[-1] if not historical_df.empty else None
    real_time_df = calculate_daily_returns(real_time_df, prev_close=last_close)
    real_time_df['source'] = 'real-time'
    updated_historical_outliers_df, updated_real_time_outliers_df = update_outliers_list(real_time_df, historical_outliers_df, pd.DataFrame())
    # Update historical data
    historical_df = pd.concat([historical_df.iloc[1:], real_time_df])  # Keep historical data rolling
else:
    print("No new data available or failed to fetch real-time data.")
    
# Combine data for Top 10 Outliers
full_outlier_df = pd.concat([updated_historical_outliers_df, updated_real_time_outliers_df])

# Print the Outliers
full_outlier_df

In [ ]:
sorted_outliers_data = full_outlier_df.sort_values(by="date")
sorted_outliers_data

## Fetching Hourly data for 3 days prior and post outlier days

In [ ]:
# Convert dates in dataset to datetime objects
sorted_outliers_data['date'] = pd.to_datetime(sorted_outliers_data['date'])

date_ranges = pd.DataFrame({
    "start_date": sorted_outliers_data['date'] - BDay(10),
    "end_date": sorted_outliers_data['date'] + BDay(11),
    "outlier_date": sorted_outliers_data['date'],
})

date_ranges

In [ ]:
def calculate_daily_returns(df, prev_close=None):
    if prev_close is not None:
        df.loc[df.index[0], 'prev_close'] = prev_close
    else:
        df['prev_close'] = df['c'].shift(1)
    df['returns'] = (df['c'] - df['prev_close']) / df['prev_close']
    return df

def fetch_hourly_data_chunk(symbol, start_date, end_date, api_key):
    formatted_start_date = start_date.strftime('%Y-%m-%d')
    formatted_end_date = end_date.strftime('%Y-%m-%d')

    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/hour/{formatted_start_date}/{formatted_end_date}?apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: {response.status_code} - {response.text}")
        return None
    
    response_data = response.json()
    
    if 'results' not in response_data:
        print(f"No 'results' in response: {response_data}")
        return None

    df = pd.DataFrame(response_data['results'])
    df['date'] = pd.to_datetime(df['t'], unit='ms')
    df.drop(columns=['t'], inplace=True)
    
    return df

def fetch_and_process_hourly_data(symbol, start_date, end_date, api_key):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    
    # Split the date range into smaller chunks
    chunk_size = 3  # Fetch data in 7-day chunks
    date_ranges = [(start_date + timedelta(days=i*chunk_size), 
                    min(end_date, start_date + timedelta(days=(i+1)*chunk_size - 1)))
                   for i in range((end_date - start_date).days // chunk_size + 1)]

    # print((end_date - start_date).days // chunk_size + 1)
    all_data = []

    for start, end in date_ranges:
        chunk_data = fetch_hourly_data_chunk(symbol, start, end, api_key)
        if chunk_data is not None:
            all_data.append(chunk_data)
    
    if not all_data:
        print("No data fetched")
        return None
    
    df = pd.concat(all_data)
    hourly_data = calculate_daily_returns(df)
    hourly_data.set_index('date', inplace=True)
    
    full_index = pd.date_range(start=start_date, end=end_date + timedelta(days=1), freq='H')
    hourly_data = hourly_data.reindex(full_index)
    
    hourly_data.reset_index(inplace=True)
    hourly_data.rename(columns={'index': 'date'}, inplace=True)
    
    return hourly_data


In [ ]:
# # Initialize empty DataFrame
# all_data = pd.DataFrame()

# # Initialize an outlier identifier starting from 1 or any specific number
# outlier_id = 1

# for index, row in date_ranges.iterrows():
#     # Convert start_date, end_date, and outlier_date to Timestamp for consistent comparison
#     start_date_ts = pd.Timestamp(row['start_date'])
#     end_date_ts = pd.Timestamp(row['end_date']) + pd.Timedelta(days=1)  # Extend the end date by one additional day
#     outlier_date_ts = pd.Timestamp(row['outlier_date'])
    
#     # Get hourly data for the range including 3 days before and after the outlier
#     hourly_data = fetch_and_process_hourly_data(symbol, start_date_ts, end_date_ts, api_key)
#     # print(hourly_data)
#     # Check if hourly_data is not None before processing
#     if hourly_data is not None:
#         # Assign the current outlier_id to the data
#         hourly_data['outlier_id'] = outlier_id

#         # Filter out weekdends
#         hourly_data = hourly_data[~hourly_data['date'].dt.weekday.isin([5,6])]
        
#         # prior_data from start_date to outlier_date inclusive
#         prior_data = hourly_data[(hourly_data['date'] >= start_date_ts) & (hourly_data['date'] < outlier_date_ts)]
#         prior_data["day type"] = "prior day"

#         # outlier_data is for the hourly data on the day of the outlier
#         outlier_data = hourly_data[(hourly_data['date'].dt.date == outlier_date_ts.date())]
#         outlier_data["day type"] = "outlier day"
        
#         # post_data from the day after outlier_date to end_date
#         post_outlier_ts = outlier_date_ts + pd.Timedelta(days=1)  # Starting the day after the outlier_date
#         post_data = hourly_data[(hourly_data['date'] > post_outlier_ts) & (hourly_data['date'] <= end_date_ts)]
#         post_data["day type"] = "post day"
#         # print(post_outlier_ts)
#         # print(end_date_ts)
#         # print(post_data)
        
#         # Concatenate the data from this iteration to the cumulative DataFrame
#         all_data = pd.concat([all_data, prior_data, outlier_data, post_data])

#         # Increment the outlier_id for the next iteration
#         outlier_id += 1

#     else:
#         print(f"Data not available for symbol {symbol} from {row['start_date'].date()} to {row['end_date'].date()}")

# # Add the day column to the final DataFrame
# all_data['day'] = all_data['date'].dt.day_name()
# # Optionally, you can reset the index of the final DataFrame if it becomes non-unique after concatenations
# all_data.reset_index(drop=True, inplace=True)

## Calculating and comparing similarity scores

In [ ]:
# def dtw_distance(series1, series2):
#     n, m = len(series1), len(series2)
#     dtw_matrix = np.zeros((n+1, m+1))
    
#     for i in range(n+1):
#         for j in range(m+1):
#             dtw_matrix[i, j] = np.inf
#     dtw_matrix[0, 0] = 0
    
#     for i in range(1, n+1):
#         for j in range(1, m+1):
#             cost = abs(series1[i-1] - series2[j-1])
#             # Take last min from a square sub-matrix
#             last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
#             dtw_matrix[i, j] = cost + last_min
            
#     return dtw_matrix[n, m]

# # Load your data
# data = pd.read_csv('USDCHF_hourly.csv')
# # Fill null values with ffill then bfill to ensure all nulls are handled
# data.fillna(method='ffill', inplace=True)
# data.fillna(method='bfill', inplace=True)
# all_data.fillna(method='ffill', inplace=True)
# all_data.fillna(method='bfill', inplace=True)

# # Filter Prior Day Data
# prior_day_data = data[data['day type'] == 'prior day']

# # # Main series to compare others against
# main = all_data[all_data['day type'] == 'prior day']
# ts_main_id = main[main['outlier_id'] == 10]['c'].values

# # Calculate DTW distances between the current outlier's series and each of the other outliers
# dtw_distances = {}
# for id in prior_day_data["outlier_id"].unique():
#     series = prior_day_data[prior_day_data['outlier_id'] == id]['c'].values
#     distance = dtw_distance(ts_main_id, series)
#     dtw_distances[id] = distance

# # Sort the distances and get the top 10 lowest
# top_10_ids = sorted(dtw_distances, key=dtw_distances.get)[:10]

# lowest_dtw = 99999

# # Output the results
# for id in top_10_ids:
#     print(f"DTW Distance between Current Outlier and ID {id}: {dtw_distances[id]:.2f}")
#     if dtw_distances[id] < lowest_dtw:
#         lowest_dtw = dtw_distances[id]
#         best_outlier_match_id = id
#     # best_outlier_match_id = min(best_outlier_match_id, dtw_distances[id])

# print(best_outlier_match_id)
# data_aug = data[data['outlier_id'] == best_outlier_match_id]
# print(len(data_aug))

In [ ]:
# data_aug.drop(columns=["Unnamed: 0", "n", "prev_close", "returns", "outlier_id", "day"], inplace=True)
# # Reset the index
# data_aug.reset_index(drop=True, inplace=True)

# data_aug

## Augmenting Data for the choosen historical outlier

In [ ]:
# # Define the autoencoder model
# def create_autoencoder(input_dim):
#     input_layer = Input(shape=(input_dim,))
#     encoded = Dense(64, activation='relu')(input_layer)
#     encoded = Dense(32, activation='relu')(encoded)
#     decoded = Dense(64, activation='relu')(encoded)
#     output_layer = Dense(input_dim)(decoded)
#     autoencoder = Model(input_layer, output_layer)
#     autoencoder.compile(optimizer='adam', loss='mse')
#     return autoencoder

# # Prepare the data
# scaler = MinMaxScaler()
# data_scaled = scaler.fit_transform(data_aug[['c']])

# # Train the autoencoder
# autoencoder = create_autoencoder(data_scaled.shape[1])
# autoencoder.fit(data_scaled, data_scaled, epochs=100, batch_size=32, shuffle=True, verbose=1)

# # Generate synthetic data
# synthetic_data = []
# for _ in range(50):
#     noise = np.random.normal(0, 0.1, data_scaled.shape)
#     synthetic_data_scaled = autoencoder.predict(data_scaled + noise)
#     synthetic_data.append(scaler.inverse_transform(synthetic_data_scaled))

In [ ]:
# # Initialize an outlier identifier starting from 1 or any specific number
# outlier_id = 1

# # Initialize an empty list to hold the individual synthetic dataframes
# synthetic_dfs = []

# # Iterate over each synthetic dataset
# for i, synthetic in enumerate(synthetic_data):
#     # Create a DataFrame from the synthetic data with a single column 'c'
#     df = pd.DataFrame(synthetic, columns=['c'])
#     # Add the 'day type' column from the existing 'data' DataFrame
#     df['day type'] = data['day type']
#     # Add the outlier_id column
#     df['outlier_id'] = outlier_id
#     # Increment the outlier_id for the next iteration
#     outlier_id += 1
#     # Append the DataFrame to the list
#     synthetic_dfs.append(df)

# # Concatenate all the individual DataFrames into a single DataFrame
# combined_df = pd.concat(synthetic_dfs, ignore_index=True)

# # Create a DataFrame from data_aug with outlier_id = 0
# data_aug_df = data_aug[['c', 'day type']].copy()
# data_aug_df['outlier_id'] = 0

# # Concatenate the synthetic DataFrames with the data_aug DataFrame
# df_train = pd.concat([data_aug_df, combined_df], ignore_index=True)

# # Rename columns to match the desired format
# df_train.columns = ['c', 'day_type', 'outlier_id']

# # Reorder the columns to have 'c', 'outlier_id', 'day_type'
# df_train = df_train[['c', 'outlier_id', 'day_type']]

In [ ]:
# df_train

## Predictive Modeling

### Fetching 3 days prior and post data for the latest outlier for vailidating predictions

In [ ]:
# # Initialize an outlier identifier starting from 1 or any specific number
outlier_id = 1

# Convert start_date, end_date, and outlier_date to Timestamp for consistent comparison
start_date_co = pd.Timestamp(date_ranges['start_date'].iloc[-1])
end_date_co = pd.Timestamp(date_ranges['end_date'].iloc[-1]) + pd.Timedelta(days=1)  # Extend the end date by one additional day
outlier_date_co = pd.Timestamp(date_ranges['outlier_date'].iloc[-1])

# Get hourly data for the range including 3 days before and after the outlier
hourly_data = fetch_and_process_hourly_data(symbol, start_date_co, end_date_co, api_key)

# Assign the current outlier_id to the data
hourly_data['outlier_id'] = outlier_id

# Filter out weekdends
hourly_data = hourly_data[~hourly_data['date'].dt.weekday.isin([5,6])]

# prior_data from start_date to outlier_date inclusive
df_prior = hourly_data[(hourly_data['date'] >= start_date_co) & (hourly_data['date'] < outlier_date_co)]
df_prior["day type"] = "prior day"

# outlier_data is for the hourly data on the day of the outlier
df_outlier = hourly_data[(hourly_data['date'].dt.date == outlier_date_co.date())]
df_outlier["day type"] = "outlier day"

# post_data from the day after outlier_date to end_date
post_outlier_co = outlier_date_co + pd.Timedelta(days=1)  # Starting the day after the outlier_date
df_post = hourly_data[(hourly_data['date'] >= post_outlier_co) & (hourly_data['date'] < end_date_co)]
df_post["day type"] = "post day"

new_df = pd.concat([df_prior, df_outlier, df_post], axis=0)

new_df

### Model training and prediction

In [ ]:
# # Function to prepare sequences for training
# def prepare_sequences(df):
#     sequences = []
#     labels = []
    
#     unique_ids = df['outlier_id'].unique()
    
#     for oid in unique_ids:
#         prior_data = df[(df['outlier_id'] == oid) & (df['day_type'].isin(['prior day', 'outlier day']))]['c'].values
#         post_data = df[(df['outlier_id'] == oid) & (df['day_type'] == 'post day')]['c'].values
        
#         if len(prior_data) >= 72 and len(post_data) >= 72:  # Ensure full sequences
#             sequences.append(prior_data[-72:])
#             labels.append(post_data[:72])  # Only the first 3 post days are needed
            
#     return np.array(sequences), np.array(labels)

# def train_model(df_train):
# 	df_train.fillna(method='bfill', inplace=True)
# 	scaler = MinMaxScaler(feature_range=(0, 1))
# 	df_train['c'] = scaler.fit_transform(df_train[['c']])

# 	X_train, y_train = prepare_sequences(df_train)
# 	if X_train.size == 0:
# 		return None, None  # Early exit if no training data

# 	X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# 	model = Sequential([
# 		LSTM(50, return_sequences=True, input_shape=(72, 1)),
# 		LSTM(50),
# 		Dense(72)
# 	])
# 	model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
# 	model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=2)
# 	return model, scaler

# # Function to prepare a single sequence for prediction
# def prepare_sequence_predict(df_prior, scaler):
# 	df_prior.drop(columns= ['prev_close', 'returns', 'n'], inplace=True)
# 	df_prior.fillna(method='bfill', inplace=True)
# 	prior_data = df_prior['c'].values # "df_prior" should be the dataframe containing prior day's data for the current outlier.
# 	prior_data = scaler.transform(prior_data.reshape(-1, 1)).flatten()
# 	return prior_data.reshape(1, 72, 1)

# # Function to predict using the model
# def predict(model, df_prior, scaler):
#     X_test = prepare_sequence_predict(df_prior, scaler)
#     predictions = model.predict(X_test)
#     return scaler.inverse_transform(predictions.reshape(-1, 1))

# # Function to validate predictions
# def validate_predictions(predictions, df_post, scaler):
# 	#df_post.drop(columns= ['prev_close', 'returns', 'n'], inplace=True)
# 	df_post.fillna(method='ffill', inplace=True)
# 	post_data = df_post['c'].values[:72] # "df_post" should be the dataframe containing post day's data for the current outlier.
# 	post_data_scaled = scaler.transform(post_data.reshape(-1, 1)).flatten()

# 	mse = mean_squared_error(post_data_scaled, predictions.flatten())
# 	mae = mean_absolute_error(post_data_scaled, predictions.flatten())
# 	rmse = np.sqrt(mse)
# 	r2 = r2_score(post_data_scaled, predictions.flatten())
	
# 	return mse, mae, rmse, r2

# # Example usage:
# model, scaler = train_model(df_train)
# predictions = predict(model, df_prior, scaler)
# mse, mae, rmse, r2 = validate_predictions(predictions, df_post, scaler)

# # Now you can print these values in a different cell
# print(f"Mean Squared Error: {mse}")
# print(f"Mean Absolute Error: {mae}")
# print(f"Root Mean Squared Error: {rmse}")
# print(f"R-squared: {r2}")

Mean Squared Error: 15.63521443905872

Mean Absolute Error: 3.953213237589056

Root Mean Squared Error: 3.954138899818609

R-squared: -2287.3112314300106

# Time Series Generator

In [ ]:
sorted_df = new_df.sort_values(by='date', ascending=True)
sorted_df.fillna(method='bfill', inplace=True)
sorted_df.fillna(method='ffill', inplace=True)

sorted_df

In [ ]:
# Split the dataset
train_set = sorted_df.iloc[:264].reset_index(drop=True)
test_set = sorted_df.iloc[264:].reset_index(drop=True)

# Reset index if needed
train_set = train_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

In [ ]:
sequence_length = 240  # Example sequence length for LSTM model - 10 days - 240 hours

# Filter data for the single outlier_id
outlier_id = train_set["outlier_id"].unique()[0]
train_df = train_set[train_set["outlier_id"] == outlier_id].set_index("date")
train_df.index = pd.to_datetime(train_df.index)  # Convert index to DateTimeIndex
test_df = test_set[test_set["outlier_id"] == outlier_id]

# Normalize data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df[["c"]])

# Prepare data for LSTM model
train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=sequence_length, batch_size=1)

# Define and compile LSTM model
model = Sequential([
    LSTM(64, activation='relu', input_shape=(sequence_length, 1)),
    Dense(1)
])
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Fit the model
model.fit(train_generator, epochs=100, verbose=0)

# Prepare last sequence for forecasting
last_sequence = train_scaled[-sequence_length:]

# Iteratively forecast the next 240 steps
forecast_steps = 240
predictions_scaled = []
for _ in range(forecast_steps):
    # Reshape the last sequence for prediction
    last_sequence_reshaped = last_sequence.reshape((1, sequence_length, 1))
    # Predict the next step and append to predictions
    next_step_pred = model.predict(last_sequence_reshaped, verbose=0)
    predictions_scaled.append(next_step_pred.ravel()[0])
    # Update the last sequence with the prediction
    last_sequence = np.roll(last_sequence, -1)
    last_sequence[-1] = next_step_pred

# Inverse transform predictions
predictions_inv = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))

# Compute metrics for the single outlier_id
actuals = test_df["c"].values[:forecast_steps]

mse = mean_squared_error(actuals, predictions_inv)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions_inv)
mape = mean_absolute_percentage_error(actuals, predictions_inv)
r2 = r2_score(actuals, predictions_inv)

# Calculate sMAPE
smape = np.mean(2 * np.abs(predictions_inv - actuals) / (np.abs(predictions_inv) + np.abs(actuals))) * 100

# Print metrics
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAPE:", mape)
print("sMAPE:", smape)
print("R^2:", r2)

In [ ]:
predictions_inv

In [ ]:
prediction = pd.DataFrame(predictions_inv)

prediction.to_csv('3.csv')
sorted_df.to_csv('4.csv')

#### .iloc(-2)
1. MAE: 0.01127558805931891
2. MSE: 0.00015902518543765831
3. RMSE: 0.012610518840938239
4. MAPE: 0.013025547450473286
5. sMAPE: 1.3092876639982067
6. R^2: -3.1421988118014763




## Results

#### 3 step prediction - 3 hours
1. RMSE: 0.0014249278565365072
2. MAE: 0.0013819148000081178
3. MAPE: 0.0016188677009796257
4. sMAPE: 0.16202621303027964
5. R^2: -152.04668816591115

#### 24 steps predictions - 24 hours - 1 day
1. RMSE: 0.0034528471986093857
2. MAE: 0.0031602989514668786
3. MAPE: 0.0037021375133191553
4. sMAPE: 0.38530215208360313
5. R^2: -3.3388885069594343

#### 216 steps - 9 days
1. MAE: 0.003084491074504933
2. MSE: 1.2475740906942047e-05
3. RMSE: 0.0035321014859346902
4. MAPE: 0.0036341938240416825
5. sMAPE: 0.4224267720020953
6. R^2: 0.11754503942796934

#### 240 steps - 10 days
1. MAE: 0.002961868462021624
2. MSE: 1.2733241896113776e-05
3. RMSE: 0.003568366838781262
4. MAPE: 0.0034965434280448044
5. sMAPE: 0.36766986776281707
6. R^2: 0.07253019544913986

#### 336 steps - 14 days
1. MAE: 0.009938953848609888
2. MSE: 0.00013717247923436177
3. RMSE: 0.011712065540901049
4. MAPE: 0.011490952565699
5. PE: 1.1586982793510603
6. R^2: -2.244672531975713

## Evaluation Parameters

### Eval Params for Time Series Forecasting

In time series forecasting, the following evaluation metrics are commonly used to assess model performance:

1. **Mean Absolute Error (MAE):**
   - Measures the average magnitude of the errors between predicted and actual values, regardless of direction.
   - Formula: 
	 
     $\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} \left| y_i - \hat{y}_i \right|$

   - **Pros:** Easy to interpret.
   - **Cons:** Treats all errors equally, without penalizing larger errors more.


2. **Mean Squared Error (MSE):**
   - Measures the average of the squared errors, which penalizes larger errors more heavily.
   - Formula: 

     $\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2$

   - **Pros:** Highlights larger errors due to squaring, which can be useful if large deviations are particularly undesirable.
   - **Cons:** Can be sensitive to outliers.

3. **Root Mean Squared Error (RMSE):**
   - The square root of MSE, bringing the error metric back to the original scale of the data.
   - Formula: 

     $\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2}$

   - **Pros:** More interpretable than MSE due to being on the same scale as the data.
   - **Cons:** Still sensitive to outliers.

4. **Mean Absolute Percentage Error (MAPE):**
   - Measures the average percentage error between the forecasted and actual values.
   - Formula: 

     $\text{MAPE} = \frac{100}{n} \sum_{i=1}^{n} \left| \frac{y_i - \hat{y}_i}{y_i} \right|$

   - **Pros:** Expresses errors as a percentage, which can be easier to interpret across different datasets.
   - **Cons:** Can be misleading if actual values are close to zero, leading to high error rates.

5. **Symmetric Mean Absolute Percentage Error (sMAPE):**
   - A variation of MAPE that addresses its sensitivity to extreme values.
   - Formula: 

     $\text{sMAPE} = \frac{100}{n} \sum_{i=1}^{n} \frac{\left| y_i - \hat{y}_i \right|}{\left( |y_i| + |\hat{y}_i| \right) / 2}$

   - **Pros:** Less sensitive to large percentage errors.
   - **Cons:** More complex to interpret.

6. **R-squared (Coefficient of Determination):**
   - Measures the proportion of the variance in the dependent variable that is predictable from the independent variables.
   - Formula:

     $R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$

   - **Pros:** Indicates the goodness-of-fit.
   - **Cons:** Not always informative for time series data, as it assumes independence between observations.

7. **Mean Absolute Scaled Error (MASE):**
   - Compares the forecasting model’s performance to a naïve forecast (e.g., last observed value).
   - Formula: 

     $\text{MASE} = \frac{\text{MAE}}{\text{MAE of naïve forecast}}$

   - **Pros:** Useful for comparing models across different datasets.
   - **Cons:** Requires a good benchmark model for comparison.

In our case of predicting close prices, MAE, RMSE, and MASE are typically the most relevant because they directly measure the error magnitude in the same units as the data, making them more interpretable for financial time series.

### Eval Params for Financial Market

In addition to the standard metrics, the following metrics can provide deeper insights into the performance of our time series forecasting model in the forex market:

1. **Directional Accuracy (DA):**
   - Measures how often the model correctly predicts the direction of price movement (up or down).
   - Formula:

     $\text{DA} = \frac{1}{n} \sum_{i=1}^{n} \mathbb{1}\left[\text{sign}(\hat{y}_i - y_{i-1}) = \text{sign}(y_i - y_{i-1})\right]$

   - **Pros:** Directly relevant for trading strategies, as correct direction prediction is often more critical than exact value prediction.
   - **Cons:** Does not account for the magnitude of the forecast error.

2. **Profit and Loss (P&L):**
   - Evaluates the hypothetical profit or loss if a trading strategy based on the model's predictions was implemented.
   - **Pros:** Provides practical insights into how well the model would perform in real trading scenarios.
   - **Cons:** Requires a well-defined trading strategy to simulate P&L.

3. **Hit Rate (HR):**
   - Measures the percentage of times the model's predicted direction and the actual market direction match.
   - **Pros:** Useful for understanding how often the model makes correct directional calls.
   - **Cons:** Does not quantify the magnitude of errors when the direction is incorrect.

4. **Maximum Drawdown (MDD):**
   - Measures the largest peak-to-trough decline in a simulated trading equity curve using the model's predictions.
   - **Pros:** Helps assess the risk of the model in terms of potential losses.
   - **Cons:** Requires simulation over a trading strategy to calculate.

5. **Sharpe Ratio:**
   - Evaluates the risk-adjusted return of a trading strategy based on the model's predictions.
   - Formula:

     \text{Sharpe Ratio} = \frac{\text{Average P\&L} - \text{Risk-Free Rate}}{\text{Standard Deviation of P\&L}}

   - **Pros:** Balances returns with the associated risk, offering a comprehensive measure of strategy performance.
   - **Cons:** Assumes returns are normally distributed, which might not always hold in financial markets.

6. **Mean Directional Accuracy (MDA):**
   - Similar to directional accuracy but gives a more refined measure by focusing on the average accuracy of the directional predictions.
   - Formula:

     \text{MDA} = \frac{1}{n} \sum_{i=1}^{n} \frac{\mathbb{1}\left[\text{sign}(\hat{y}_i - y_{i-1}) = \text{sign}(y_i - y_{i-1})\right]}{n}

   - **Pros:** Useful when focusing on directionality rather than magnitude.
   - **Cons:** Overlooks large errors in magnitude.

7. **Volatility Adjusted Return (VAR):**
   - Measures return adjusted for volatility to understand how the model performs under varying market conditions.
   - **Pros:** Important in markets like forex, where volatility can significantly impact performance.
   - **Cons:** Requires more complex calculation and an understanding of market volatility.

8. **Value at Risk (VaR):**
   - Estimates the potential loss in value of a portfolio over a defined period for a given confidence interval.
   - **Pros:** Helps assess the potential downside risk.
   - **Cons:** Does not account for losses beyond the confidence interval threshold.

Using these metrics in combination with the standard error metrics can give us a more comprehensive understanding of how our model might perform in real-world trading scenarios, particularly in the highly volatile forex market.